In [1]:
from semantic_router.splitters import RollingWindowSplitter
from semantic_router.utils.logger import logger
from semantic_router.encoders import HuggingFaceEncoder
import numpy as np
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
import os

from unstructured.partition.md import partition_md

load_dotenv()

def process_mdx_advanced(file_path):
    # partition_md tự động xử lý frontmatter và cấu trúc markdown
    elements = partition_md(filename=file_path)
    
    # Gộp các element lại thành văn bản
    full_text = "\n\n".join([str(el) for el in elements])
    return full_text


/home/sunny/workspace/CP-Helper/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Pinecone
from langchain_ollama import ChatOllama
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

logger.setLevel("WARNING")

embeddings = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1.5",
    model_kwargs={'trust_remote_code': True},
    encode_kwargs={
        'normalize_embeddings': True,  # Semantic Router does this by default
    }
)
vector_store = PineconeVectorStore(
    pinecone_api_key=os.getenv("PINECONE_API_KEY"),
    embedding=embeddings,
    index_name='cp-rag',
    text_key="content"
)

retriever = vector_store.as_retriever(kwargs={"k": 5})

template="""You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)
generate_queries = (
    prompt_perspectives 
    | ChatOllama(model="llama3") 
    | StrOutputParser() 
    | (lambda x: [
        line.strip()                            # Remove whitespace
        for line in x.split("\n") 
        if line.strip() and not line.strip().isnumeric() # Remove empty lines and stray numbers
      ])
)

/home/sunny/workspace/CP-Helper/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
<All keys matched successfully>
/tmp/ipykernel_5345/2169286346.py:34: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  | ChatOllama(model="llama3")


In [3]:
print(len(embeddings.embed_query("test")))

768


In [4]:
docs = retriever.invoke("What is Segment Tree?")
for res in docs:
    print(f"* {res.page_content} [{res.metadata}]")

* Resources Sparse Segment Tree is more commonly referred to as "Dynamic Segment Tree." Monkey and Apple-trees Solution We need to support two operations on a range: Count the number of red-ripe trees in a range (range sum) Set all trees in a range to red-ripe (range paint) We can use a segment tree with lazy propagation to solve this, but the query range is up to $10^9$, so we have to use a sparse segment tree. Luckily, lazy propagation still works here. [{'postchunk': 'Pointer Implementation Time Complexity: $\\mathcal{O}(Q\\log{N})$ #include <bits/stdc++.h> using namespace std; class SparseSegtree { private: struct Node { int freq = 0; int lazy = 0;', 'prechunk': "id: sparse-segtree redirects: - /plat/sparse-seg title: Sparse Segment Trees author: Andi Qu prerequisites: - RURQ description: 'Querying big ranges.' frequency: 1 In problems where the query range is at most something like $10^6$, a normal segment tree suffices. However, as soon as we move to bigger ranges ($10^{12}$ in s

In [10]:
def get_unique_union(documents: list[list]):
    """ Flattens a list of lists of documents and removes duplicates. """
    flattened = [doc for sublist in documents for doc in sublist]
    unique_docs = {}
    for doc in flattened:
        # Use page_content as the unique key
        if doc.page_content not in unique_docs:
            unique_docs[doc.page_content] = doc
    return list(unique_docs.values())

def retrieve_documents(question: str):
    # generate_queries returns List[str]
    # retriever.map() expects List[str] and returns List[List[Document]]
    # get_unique_union expects List[List[Document]] and returns List[Document]
    retrieval_chain = generate_queries | retriever.map() | get_unique_union
    
    # Passing the dictionary matches the input expectation of prompt_perspectives
    docs = retrieval_chain.invoke({"question": question})
    return docs


In [6]:
docs = retrieve_documents("What is Binary Search?")
for doc in docs:
    print(doc)

Here are five different versions of the original question:
What is the concept behind Binary Search?
How does Binary Search algorithm work in computer science?
Can you explain the process of Binary Search in a simple way?
What are the key features of Binary Search technique?
Is there an analogy or metaphor that illustrates how Binary Search works?
These alternative questions aim to capture different aspects and perspectives on the original question, which can help retrieve relevant documents from the vector database.


In [7]:
print(docs)

for question in docs:
    print(question)
    print(len(embeddings.embed_query(question)))
    temp = retriever.invoke(question)
    for res in temp:
        print(f"* {res.page_content} [{res.metadata}]")

['Here are five different versions of the original question:', 'What is the concept behind Binary Search?', 'How does Binary Search algorithm work in computer science?', 'Can you explain the process of Binary Search in a simple way?', 'What are the key features of Binary Search technique?', 'Is there an analogy or metaphor that illustrates how Binary Search works?', 'These alternative questions aim to capture different aspects and perspectives on the original question, which can help retrieve relevant documents from the vector database.']
Here are five different versions of the original question:
768
* Q: Do I need to participate in the USACO December contest before participating in the other contests? No. Q: How are USACO contests scored? USACO contests are scored out of $1000$ points. Each problem is worth $1000/3 = 333.333 \dots$ points. When a solution to a problem is submitted, it is run against a number of test cases (see the second link above for details). All test cases for a p

In [11]:
def query(question: str):
    context = ""

    docs = retrieve_documents(question)
    for match in docs:
        title = match.metadata['title']
        pre = match.metadata['prechunk']
        content = match.page_content
        post = match.metadata['postchunk']

        context += '\n--------\nTitle: '
        context += title
        context += '\n'
        context += pre
        context += '\n'
        context += content
        context += '\n'
        context += post
        context += '\n'

    full_prompt = f'''
    You are a extremely helpful assistant in Competittive Programming. 
    Use the info in the context to answer the question as much detailed as possible. 
    If there are any code, implement it fully and dont hide any line.

    If you cannot answer, just say "I don't know"

    Context:
    {context}

    Question:
    {question}
    '''

    import ollama

    try:
        response = ollama.chat(
            model='llama3',
            messages=[{'role': 'user', 'content': full_prompt}],
            stream=True
        )
        
        full_content = ""
        for chunk in response:
            content = chunk['message']['content']
            print(content, end='', flush=True)
            full_content += content
        print()
        
    except Exception as e:
        print(f"Lỗi khi gọi model: {e}")

In [12]:
query("What is Segment Tree?")

A great question!

A Segment Tree is a data structure used to efficiently update and query ranges of elements in an array or list. It's commonly used in competitive programming and computer science to solve problems involving range queries, such as finding the sum or minimum value in a range, updating values in a range, and more.

The basic idea behind a Segment Tree is to create

KeyboardInterrupt: 

In [ ]:
query("cách cài đặt DP convex hull trong C++. Hãy trả lời bằng tiếng việt")

Cài đặt Dynamic Programming (DP) để tìm convex hull của một tập điểm trong C++ có thể được thực hiện như sau:

Tạo ra một mảng động để lưu trữ các điểm:
```c
vector<Point> points;
```
Sau đó, tạo ra một mảng để lưu trữ kết quả convex hull:
```c
vector<Point> hull;
```
Hàm `monotoneChain` sẽ được sử dụng để tính convex hull. Hàm này sẽ nhận vào một mảng các điểm và trả về một mảng chứa các điểm trong convex hull.

Here's the implementation of the `monotoneChain` function:
```c
vector<Point> monotoneChain(vector<Point>& P) {
  if (P.size() < 3) return P; // nếu số lượng điểm dưới 3, thì toàn bộ tập điểm là convex hull

  sort(P.begin(), P.end()); // sắp xếp các điểm theo trục x và y

  vector<Point> hull;
  Point last = P[0]; // lấy điểm đầu tiên trong mảng
  hull.push_back(last);

  for (int i = 1; i < P.size(); i++) {
    if (last.x > P[i].x) { // nếu điểm hiện tại nằm bên phải của điểm cuối cùng trong hull, thì có thể thêm điểm này vào hull
      last = P[i]; // cập nhật điểm cuối cùn